In [ ]:
pip install transformers beautifulsoup4 requests

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [ ]:
from huggingface_hub import login

hf_token = os.environ.get('HF_TOKEN')

In [ ]:
# Function to extract company information from a URL
def get_company_data(url):
    # Send a GET request to the URL
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f"Failed to retrieve webpage: {url}")

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the title of the company (assuming it's in a <title> tag)
    title = soup.title.string if soup.title else "Unknown Title"

    # Extract the description and other relevant information
    # This depends on the structure of the webpage; here we use generic examples
    # Adjust according to the page's HTML structure

    # Example of extracting description (from meta tags)
    description = soup.find('meta', attrs={'name': 'description'}) or soup.find('meta', attrs={'property': 'og:description'})
    description = description['content'] if description else "No description available."

    # Example of extracting content (using paragraphs or div tags)
    content = " ".join([p.text for p in soup.find_all('p')])

    return {
        'title': title,
        'description': description,
        'content': content
    }


In [ ]:
# Function to generate text using GPT-Neo
def generate_article(company_data):
    # Initialize the GPT-Neo model
    generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

    # Create the prompt using the company data
    prompt = (
        f"Create a concise, informative article about the following company:\n\n"
        f"Title: {company_data['title']}\n"
        f"Description: {company_data['description']}\n"
        f"Content: {company_data['content']}\n\n"
        f"Summarize this information for a sales team."
    )

    # Generate the text using GPT-Neo
    result = generator(prompt, max_new_tokens=100, num_return_sequences=1, truncation=True)

    # Extract and return the generated text
    generated_text = result[0]['generated_text']
    return generated_text



In [ ]:
# Function to save generated article to a .txt file
def save_to_txt(generated_text, filename="generated_article.txt"):
    with open(filename, 'w') as file:
        file.write(generated_text)
    print(f"Article saved to {filename}")



In [ ]:
# URL of the company website
url = 'https://www.shoreprosthetics.com/services'  # Replace with the actual company URL

# Get the company data from the URL
company_data = get_company_data(url)

# Generate the article
informative_article = generate_article(company_data)

# Print the informative article
print(informative_article)

# Save the informative article to a .txt file
save_to_txt(informative_article)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Create a concise, informative article about the following company:

Title: Services – Shore Prosthetics and Orthotics – USVI
Description: No description available.
Content: Services Whether you’re looking for the newest hi-tech device or just need an evaluation of your current prosthetic device, have one of our local clinicians help you get back to doing what you love! We believe that the best devices are a blend of both art and science. Or local clinical orthotic services will help you get back to your daily activities in no time! If you need an orthotic device for any condition, give us a call to set up an appointment. Our clinicians have amassed much knowledge and experience in treating most pediatric conditions requiring orthotic intervention. From cranial remolding helmet therapy for plagiocephaly and non-surgical scoliosis bracing treatment, to AFOs, KAFOs, HKAFOs for lower extremity as well as upper extremity bracing solutions, we are dedicated to enriching your child’s interact